In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import pandas as pd
import time
from datetime import datetime
import numpy as np
import os


In [2]:
torneo = "liga-dimayor-finalizacion"

In [3]:
partidos = pd.DataFrame(columns=['year', 'torneo', 'jornada', 'local', 'goles_local', 'goles_visitante', 'visitante', 'fecha', 'dia', 'arbitro', 'pos_local', 'pos_visitante', 'gana_local', 'empate', 'gana_visitante'])


In [4]:

def scrap_page(url, year, jornada):

    # Create a WebDriver instance for Chrome
    # os.environ["webdriver.chrome.driver"] = "C:\path\to\chromedriver.exe"
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    # Visit the website
    # driver.get(url)

    # Click on accept cookies policy
    # time.sleep(2)
    # WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "sp_message_iframe_851961")))
    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Aceptar']"))).click()
    # time.sleep(2)

    driver.get(url)
    div_element = driver.find_element(By.CSS_SELECTOR, 'div.large-8.columns')
    boxes = div_element.find_elements(By.XPATH, 'div[@class="box"] [@style="border-top: 0 !important;"]')
    print(len(boxes))
    
    for i in range(len(boxes)):
        l = [year, torneo, jornada]
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[1]/a''').get_attribute("innerText")) # home
        l.append(int(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[5]/span/a/span''').get_attribute("innerText").split(":")[0])) # goals home
        l.append(int(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[5]/span/a/span''').get_attribute("innerText").split(":")[1])) # goals visitor
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[8]/a''').get_attribute("innerText")) # visitor
        l.append(datetime.strptime(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[2]/td/a''').get_attribute("href").split("/")[-1], "%Y-%m-%d").date()) # date
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[2]/td''').get_attribute("innerText").split(",")[0]) # day
        try:
            l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[3]/td''').get_attribute("innerText").split("·")[-1].strip()) # referee
        except NoSuchElementException:
            l.append(np.nan) # referee
        try:
            l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[1]/span''').get_attribute("innerText").replace("(","").replace(".)","")) # pos home
        except NoSuchElementException:
            l.append(np.nan) # referee
        try:
            l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[8]/span''').get_attribute("innerText").replace("(","").replace(".)","")) # pos visitor
        except NoSuchElementException:
            l.append(np.nan) # referee        
        if l[4] > l[5]:
            l=l+[True,False,False]
        elif l[4] == l[5]:
            l=l+[False,True,False]
        else:
            l=l+[False,False,True]
        partidos.loc[len(partidos)] = l
    
    driver.quit()
    return partidos

In [5]:
def get_cantidad(url):
    # Create a WebDriver instance for Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    # Visit the website
    driver.get(url)

    # Click on accept cookies policy
    # time.sleep(2)
    # WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "sp_message_iframe_851961")))
    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Aceptar']"))).click()
    # time.sleep(2)

    driver.get(url)
    selector = driver.find_elements(By.CSS_SELECTOR, 'ul.chzn-results')[1]
    fechas = selector.find_elements(By.XPATH, './*')
    cantidad = len(fechas)
    driver.quit()
    return cantidad

In [6]:
for i in range(2023,2024):
    # website = f"https://www.transfermarkt.es/liga-dimayor-finalizacion/spieltag/wettbewerb/COL1/spieltag/1/saison_id/{i-1}"
    # cantidad = get_cantidad(website)
    # print("año ", i, ": ", cantidad, " fechas")
    for j in range(18,20+1):
        website = f"https://www.transfermarkt.es/liga-dimayor-finalizacion/spieltag/wettbewerb/COL1/spieltag/{str(j)}/saison_id/{i-1}"
        print(website)
        partidos.append(scrap_page(website, i, j))
partidos #.to_csv("transfer.csv", mode='a', index=False, header=False)

https://www.transfermarkt.es/liga-dimayor-finalizacion/spieltag/wettbewerb/COL1/spieltag/18/saison_id/2022
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_1601/3860779578.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


https://www.transfermarkt.es/liga-dimayor-finalizacion/spieltag/wettbewerb/COL1/spieltag/19/saison_id/2022
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_1601/3860779578.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


https://www.transfermarkt.es/liga-dimayor-finalizacion/spieltag/wettbewerb/COL1/spieltag/20/saison_id/2022
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_1601/3860779578.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


,year,torneo,jornada,local,goles_local,goles_visitante,visitante,fecha,dia,arbitro,pos_local,pos_visitante,gana_local,empate,gana_visitante
0,2023,liga-dimayor-finalizacion,18,Envigado,3,0,Deportivo Cali,2023-10-20,viernes,Nicolás Gallo,20,8,True,False,False
1,2023,liga-dimayor-finalizacion,18,Jaguares FC,1,0,Deportivo Pasto,2023-10-21,sábado,David Espinosa,19,9,True,False,False
2,2023,liga-dimayor-finalizacion,18,CD La Equidad,1,1,Águilas Doradas,2023-10-21,sábado,José Ortiz,14,1,False,True,False
3,2023,liga-dimayor-finalizacion,18,Indep. Santa Fe,2,3,Millonarios,2023-10-21,sábado,Éder Vergara,10,6,False,False,True
4,2023,liga-dimayor-finalizacion,18,Union Magdalena,1,2,A. Bucaramanga,2023-10-22,domingo,Carlos Betancourt,11,13,False,False,True
5,2023,liga-dimayor-finalizacion,18,Junior FC,1,0,Once Caldas,2023-10-22,domingo,Wilmar Roldán,12,16,True,False,False
6,2023,liga-dimayor-finalizacion,18,Boyacá Chicó,1,3,Atl. Nacional,2023-10-22,domingo,Jhon Ospina,18,5,False,False,True
7,2023,liga-dimayor-finalizacion,18,Indep. Medellín,1,0,Pereira,2023-10-23,lunes,Andrés Rojas,3,15,True,False,False
8,2023,liga-dimayor-finalizacion,18,Alianza Petrol.,0,1,Deportes Tolima,2023-10-23,lunes,Nolberto Ararat,7,4,False,False,True
9,2023,liga-dimayor-finalizacion,18,CD América,1,0,Atlético Huila,2023-10-24,martes,Juan Pablo Alba,2,17,True,False,False


In [12]:
partidos.to_csv("transfer.csv", mode='a', index=False, header=False)